In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from pytz import timezone
import espn_scraper as espn #https://github.com/andr3w321/espn_scraper
import json

def strzero(num):
    num = str(num)
    if len(num) == 1:
        num = '0'+num
    return(num)

In [ ]:
year = 2019
month = 9
day = 12
date = '20190822'
schedURL = 'https://www.espn.com/wnba/scoreboard/_/date/'+str(year)+strzero(month)+strzero(day)

In [ ]:
game_ids = []
game_info = []
years = list(range(1997,2022))
years = [year]
years.reverse()

In [ ]:
years

In [ ]:
#get all of the game ids ever
for year in years:
    prev_len = len(game_ids)
    scoreboard_urls= espn.get_all_scoreboard_urls('wnba', year) 
    for scoreboard_url in scoreboard_urls:
        data = espn.get_url(scoreboard_url, cached_path="cached_data")
        try:
            for event in data['content']:  
                if event['analytics']['gameId'] not in game_ids:
                    game_ids.append(event['analytics']['gameId'])
                    game_info.append([event['analytics']['gameId'],year])
        except:
            pass
    print([year,len(game_ids)-prev_len])


In [ ]:
game_id_df = pd.DataFrame(game_info)
game_id_df.columns = ['GameID','year']
game_id_df.to_csv('game_info.csv')

In [ ]:
len(game_ids)

In [ ]:
#game_ids_csv = pd.read_csv('not-scraped.csv')
GameID = '401105109'

In [ ]:
#def get_game_data(GameID):
if True:
    GameID = str(GameID)

    #scrape html
    URL = 'https://www.espn.com/wnba/playbyplay/_/gameId/'+GameID
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #extract json including pbp data
    #from: https://stackoverflow.com/questions/13323976/how-to-extract-a-json-object-that-was-defined-in-a-html-page-javascript-block-us
    script = soup.find('script', text=re.compile("window\['__espnfitt__'\]"))
    json_text = re.search(r"^\s*window\['__espnfitt__'\]\s*=\s*({.*?})\s*;\s*$",
                          script.string, flags=re.DOTALL | re.MULTILINE).group(1)
    data = json.loads(json_text)
    
    #get pbp
    pbp = data['page']['content']['gamepackage']['pbp']['playGrps']
    pbp = [item for sublist in pbp for item in sublist]
    pbp = pd.DataFrame.from_dict(pbp)
    
    #get shot chart
    shot_chart = data['page']['content']['gamepackage']['shtChrt']['plays']
    shot_chart = pd.DataFrame.from_dict(shot_chart)
    shot_chart = shot_chart[['id','athlete','coordinate','shootingPlay','type']]
    
    #combine shot chart and pbp
    combined_pbp = pd.merge(pbp,shot_chart,on='id',how='outer')
    
    #clean combined dataframe
    combined_pbp['period'] = [re.search('[0-9]',str(x)).group(0) for x in combined_pbp['period']]
    combined_pbp['clock'] = [re.search('[0-9]{1,2}([:\.])[0-9]{1,2}',str(x)).group(0) for x in combined_pbp['clock']]
    combined_pbp['player_id'] = [re.search('\d+',str(x)).group(0) if bool(re.search('\d+',str(x))) else np.nan for x in combined_pbp['athlete']]
    combined_pbp['player'] = [re.sub("{'id': '\d+', 'name': '",'',str(x)) if bool(re.search("{'id': '\d+', 'name': '",str(x))) else np.nan for x in combined_pbp['athlete']]
    combined_pbp['player'] = [re.sub("'}",'',str(x)) if bool(re.search("'}",str(x))) else np.nan for x in combined_pbp['player']]
    combined_pbp['shotType'] = [re.sub("{'id': '\d+', 'txt': '",'',str(x)) if bool(re.search("{'id': '\d+', 'txt': '",str(x))) else np.nan for x in combined_pbp['type']]
    combined_pbp['shotType'] = [re.sub("'}",'',str(x)) if bool(re.search("'}",str(x))) else np.nan for x in combined_pbp['shotType']]
    combined_pbp['shotType'] = [re.sub("{'id': '\d+', 'txt': '",'',str(x)) if bool(re.search("{'id': '\d+', 'txt': '",str(x))) else np.nan for x in combined_pbp['type']]
    combined_pbp['shotType'] = [re.sub("'}",'',str(x)) if bool(re.search("'}",str(x))) else np.nan for x in combined_pbp['shotType']]
    combined_pbp['x_coord'] = [re.findall("\d+",str(x))[0] if bool(re.search("\d+",str(x))) else np.nan for x in combined_pbp['coordinate']]
    combined_pbp['y_coord'] = [re.findall("\d+",str(x))[1] if bool(re.search("\d+",str(x))) else np.nan for x in combined_pbp['coordinate']]
    combined_pbp = combined_pbp.drop('athlete', axis=1)
    combined_pbp = combined_pbp.drop('type', axis=1)
    combined_pbp = combined_pbp.drop('coordinate', axis=1)
    
    #team info
    teams = data['page']['content']['gamepackage']['pbp']['tms']
    combined_pbp['homeTeam'] = teams['home']['abbrev']
    combined_pbp['awayTeam'] = teams['away']['abbrev']
    if 'winner' in teams['home']:
        combined_pbp['winner'] = 'home'
    elif 'winner' in teams['away']:
        combined_pbp['winner'] = 'away'
    
    #game info
    game_info = data['page']['content']['gamepackage']['gmInfo']
    combined_pbp['gameID'] = GameID
    combined_pbp['dateTime'] = datetime.datetime.strptime(game_info['dtTm'],'%Y-%m-%dT%H:%MZ') - timedelta(hours=7) #put datetime in pacific time
    combined_pbp['neutralSite'] = data['page']['content']['gamepackage']['shtChrt']['ntrlSte']
    
    #return combined_pbp

In [ ]:
df = pd.DataFrame()
gid_fails = []
#game_ids_test = [401230733]
game_ids_test = game_ids
for GameID in game_ids_test:
    try:
        temp = get_game_data(GameID) #for each game id get pbp, shot chart, game info in one table
        df = pd.concat([df,temp],ignore_index=True) #append to previous
    except:
        gid_fails.append(GameID) #if fail, print the gameid
        
print(gid_fails)

In [ ]:
df.to_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv',index=False)

In [ ]:
df

In [ ]:
df[['gameID','dateTime','homeTeam','awayTeam']].drop_duplicates().to_csv(year+"-scraped.csv")

In [ ]:
#to do 
#go back and get other game info

In [ ]:
##archive parse html code not needed with ESPN parsing as of 7/30/22
##if True:
#def get_game_data(GameID):
#    #first get all the game info
#    data_type = "recap"
#    url = espn.get_game_url(data_type, "wnba", GameID)
#    data = espn.get_url(url)
#    temp = data['gamepackageJSON']
#    temp = temp['header']   
#    #date
#    Date = temp['competitions'][0]['date']
#    #winning team
#    if temp['competitions'][0]['competitors'][0]['winner']: #check if 1st team won
#        WinningTeam = temp['competitions'][0]['competitors'][0]['team']['abbreviation']
#    else: #otehrwise 2nd team won
#        WinningTeam = temp['competitions'][0]['competitors'][1]['team']['abbreviation']
#    #regular season or playoffs?
#    gametypes = ['none','preseason','regular season','playoffs']
#    gametype_int = temp['season']['type']
#    GameType = gametypes[gametype_int]
#    
#    ########################################
#    #now get all the play-by-play
#    GameID = str(GameID)
#    URL = 'https://www.espn.com/wnba/playbyplay?gameId='+GameID
#    page = requests.get(URL)
#    soup = BeautifulSoup(page.content, 'html.parser')
#    
#    #teams
#    teams = [x.text for x in soup.find_all('span', class_='abbrev')]
#    AwayTeam = teams[0]
#    HomeTeam = teams[1]
#    teams = {teams[0]:'away',teams[1]:'home'}
#    
#    #scrape the shot description table
#    times = [x.text for x in soup.find_all('td', class_='time-stamp')]
#    plays = [x.text for x in soup.find_all('td', class_='game-details')]
#    score = [x.text for x in soup.find_all('td', class_='combined-score')]
#    
#    #get team names from the logo image urls
#    logourl = [x.find('img').get('src') for x in soup.find_all('td', class_='logo')]
#    team = [re.match(r"https.*/wnba/500/(?P<team>[a-z]+)\.",x).group('team').upper() for x in logourl]
#    homeoraway = [teams[x] for x in team]
#    
#    #convert to df
#    shotdescriptions = pd.DataFrame({'Time': times,
#                          'EventDescription': plays,
#                          'Score': score,
#                          'EventTeam': team,
#                          'HomeOrAway':homeoraway})
#    shotdescriptions['GameID'] = GameID
#    shotdescriptions['Date'] = Date
#    shotdescriptions['GameType'] = GameType
#    shotdescriptions['WinningTeam'] = WinningTeam
#    shotdescriptions['AwayTeam'] = AwayTeam
#    shotdescriptions['HomeTeam'] = HomeTeam
#    
#    
#    #get shot chart data for away team
#    awayshots = soup.find('ul', class_='shots away-team')
#    awayshots = awayshots.find_all('li')
#    awayshotchart = []
#    for row in awayshots:
#        coordtext = row.get('style')
#        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
#        xcoord = match.group('xcoord')
#        ycoord = match.group('ycoord')
#        ycoord = str(100-float(ycoord))
#        row_data = [row.get('class')[0],
#                    row.get('data-homeaway'),
#                    row.get('data-period'),
#                    row.get('data-shooter'),
#                    row.get('data-text'),
#                    row.get('id'),
#                    xcoord,
#                    ycoord]
#        awayshotchart.append(row_data)
#    
#    #get shot chart data for home team
#    homeshots = soup.find('ul', class_='shots home-team')
#    homeshots = homeshots.find_all('li')
#    homeshotchart = []
#    for row in homeshots:
#        coordtext = row.get('style')
#        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
#        xcoord = match.group('xcoord')
#        ycoord = match.group('ycoord')
#        ycoord = str(100-float(ycoord))
#        row_data = [row.get('class')[0],
#                    row.get('data-homeaway'),
#                    row.get('data-period'),
#                    row.get('data-shooter'),
#                    row.get('data-text'),
#                    row.get('id'),
#                    xcoord,
#                    ycoord]
#        homeshotchart.append(row_data)
#    
#    awayshotchart = pd.DataFrame(awayshotchart)
#    homeshotchart = pd.DataFrame(homeshotchart)
#    
#    #name columns for shot chart data
#    awayshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
#    homeshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
#    
#    #join to shot descriptions
#    playindex = 0
#    for row in range(len(awayshotchart)):
#        not_found = True
#        while not_found:
#            if shotdescriptions.loc[playindex,'EventDescription']==awayshotchart.loc[row,'EventDescription']:
#                if shotdescriptions.loc[playindex,'HomeOrAway']=='away':
#                    shotdescriptions.loc[playindex,'ShotOutcome']=awayshotchart.loc[row,'ShotOutcome']
#                    shotdescriptions.loc[playindex,'Quarter']=awayshotchart.loc[row,'Quarter']
#                    shotdescriptions.loc[playindex,'pid']=awayshotchart.loc[row,'pid']
#                    shotdescriptions.loc[playindex,'ShotID']=awayshotchart.loc[row,'ShotID']
#                    shotdescriptions.loc[playindex,'xcoord']=awayshotchart.loc[row,'xcoord']
#                    shotdescriptions.loc[playindex,'ycoord']=awayshotchart.loc[row,'ycoord']
#                    not_found = False
#            playindex += 1       
#    
#    playindex = 0
#    for row in range(len(homeshotchart)):
#        not_found = True
#        while not_found:
#            if shotdescriptions.loc[playindex,'EventDescription']==homeshotchart.loc[row,'EventDescription']:
#                if shotdescriptions.loc[playindex,'HomeOrAway']=='home':
#                    shotdescriptions.loc[playindex,'ShotOutcome']=homeshotchart.loc[row,'ShotOutcome']
#                    shotdescriptions.loc[playindex,'Quarter']=homeshotchart.loc[row,'Quarter']
#                    shotdescriptions.loc[playindex,'pid']=homeshotchart.loc[row,'pid']
#                    shotdescriptions.loc[playindex,'ShotID']=homeshotchart.loc[row,'ShotID']
#                    shotdescriptions.loc[playindex,'xcoord']=homeshotchart.loc[row,'xcoord']
#                    shotdescriptions.loc[playindex,'ycoord']=homeshotchart.loc[row,'ycoord']
#                    not_found = False
#            playindex += 1      
#    
#    #get quarter numbers and scores
#    quarter = 1
#    shotdescriptions.loc[0,'AwayScore'] = 0
#    shotdescriptions.loc[0,'HomeScore'] = 0
#    for row in range(0,len(shotdescriptions)):
#        if re.match('End of Game',shotdescriptions.loc[row,'EventDescription']) :
#            quarter -= 1 #make the end of the game the same as the last quarter
#        shotdescriptions.loc[row,'Quarter'] = quarter
#        match = re.match(("([0-9]+) - ([0-9]+)"),shotdescriptions.loc[row,'Score'])
#        shotdescriptions.loc[row,'AwayScore'] = match.group(1)
#        shotdescriptions.loc[row,'HomeScore'] = match.group(2)
#        if re.match('End',shotdescriptions.loc[row,'EventDescription']) :
#            quarter += 1 #if you reach the end of the quarter, go to the next quarter
#    
#    #return shotdescriptions

In [ ]:
#url = espn.get_game_url('playbyplay', "wnba", 401244185)
#data = espn.get_url(url)